In [1]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
cohort_id = 'BF-09_Houet_colu_2012_Q3'
cohorts_analysis = "20230223"
contigs = ['3L']
sample_sets = "3.0"
min_cohort_size = 20
max_cohort_size = 50
h12_calibration_contig = '3L'
use_gcs_cache = False
dask_scheduler = "threads"

In [12]:
from IPython.display import Markdown, HTML
import malariagen_data
import pandas as pd
from pyprojroot import here
import yaml
import dask
dask.config.set(scheduler=dask_scheduler);
from textwrap import dedent

import geopandas as gpd
import bokeh.layouts as bklay
import bokeh.plotting as bkplt
import plotly.express as px

from bokeh.io import output_notebook # enables plot interface in J notebook

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

In [3]:
# load cohorts to find sample query 
df_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson").set_index("cohort_id")
cohort = df_cohorts.loc[cohort_id]
cohort

ERROR 1: PROJ: proj_create_from_database: Open of /home/alimanfoo/envs/selection-atlas/share/proj failed


cohort_size                                                          78
country                                                    Burkina Faso
admin1_iso                                                        BF-09
admin1_name                                               Hauts-Bassins
admin2_name                                                       Houet
taxon                                                          coluzzii
year                                                               2012
quarter                                                               3
cohort_label                Burkina Faso / Houet / coluzzii / 2012 / Q3
sample_query          cohort_admin2_quarter == 'BF-09_Houet_colu_201...
latitude                                                      11.223768
longitude                                                      -4.35639
h12_window_size                                                    1000
country_alpha2                                                  

In [4]:
extra_params = dict()
if use_gcs_cache:
    extra_params["url"] = "simplecache::gs://vo_agam_release"
    extra_params["simplecache"] = dict(cache_storage=(here() / "gcs_cache").as_posix())

ag3 = malariagen_data.Ag3(
    # pin the version of the cohorts analysis for reproducibility
    cohorts_analysis=cohorts_analysis,
    results_cache=(here() / "malariagen_data_cache").as_posix(),
    **extra_params,
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : /home/alimanfoo/selection-atlas/malariagen_data_cache
Cohorts analysis        : 20230223
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 7.6.0
Client location         : unknown
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [7]:
df_samples = ag3.sample_metadata(sample_query=cohort['sample_query'])
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
1,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
2,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
3,AB0090-C,BF3-10,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
4,AB0091-C,BF3-12,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,AB0266-Cx,BF12-15,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
74,AB0267-C,BF12-16,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
75,AB0276-C,BF12-27,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
76,AB0279-C,BF12-30,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3


In [6]:
def add_empty_months(df, cohort_id):
    quarter = cohort_id[-2:]
    q_months = {'Q1':[1,2,3],
            'Q2':[4,5,6],
            'Q3':[7,8,9],
            'Q4':[10,11,12]}
    
    year = df.year.unique()
    months = df['month'].tolist()
    empty_months = list(set(q_months[quarter]) - set(months))
    
    if empty_months:
        for m in empty_months:
            df = pd.concat([df, pd.DataFrame({'year': year, 'month': m, 'count': 0})])

    return(df, quarter, year[0])

df_collection_dates = df_samples.groupby(['year', 'month']).size().reset_index().rename(columns={0: 'count'})
if df_collection_dates['month'].unique()[0] == -1:
    month_collections = False
else:
    month_collections = True
    df_collection_dates, quarter, year = add_empty_months(df_collection_dates, cohort_id)
    df_collection_dates['month'] = pd.to_datetime(df_collection_dates['month'], format='%m').dt.month_name()
    start_month = df_collection_dates.query("count > 0").month.min()
    end_month = df_collection_dates.query("count > 0").month.max()

In [9]:
df_locations = df_samples[["location", "longitude", "latitude"]].drop_duplicates()
df_locations

,location,longitude,latitude
0,Bana Village,-4.472,11.233
20,Souroukoudinga,-4.235,11.238
41,Pala,-4.235,11.151


In [35]:
df_contributors = df_samples[["release", "sample_set", "contributor"]].drop_duplicates()
df_contributors["study"] = df_contributors.apply(
    lambda v: "Ag1000G" if v["sample_set"].startswith("AG1000G") else "TODO",
    axis="columns"
)
df_contributors["release"] = df_contributors["release"].apply(
    lambda v: f"Ag{v}"
)
df_contributors.rename(columns={
    "contributor": "Contributor",
    "study": "Study",
    "release": "Data release",
    "sample_set": "Sample set",
}, inplace=True)
df_contributors.set_index(["Contributor", "Study", "Data release"], inplace=True)
df_contributors

,,,Sample set
Contributor,Study,Data release,
Austin Burt,Ag1000G,Ag3.0,AG1000G-BF-A


In [40]:
n_locations = len(df_locations)

summary_info = dedent(f"""
    This cohort comprises {cohort['cohort_size']} samples from the 
    *{cohort['taxon']}* taxon, collected from {n_locations} locations within 
    the administrative division of {cohort['admin2_name']}, 
    {cohort['admin1_name']}, {cohort['country']}.
""")

if month_collections and end_month == start_month:
    summary_info += dedent(f"""
        Collections were made in {start_month} {cohort['year']}.
    """)
elif month_collections:
    summary_info += dedent(f"""
        Collections were made between {start_month} and {end_month} 
        in {cohort['year']}.
    """)
else:
    summary_info += dedent(f"""
        Collections were made in {cohort['year']}.
    """)
    
summary_info += dedent(f"""
""")

display(Markdown(summary_info))

display(Markdown("""Samples and genomic data originate from the following studies:"""))

display(df_contributors)


This cohort comprises 78 samples from the 
*coluzzii* taxon, collected from 3 locations within 
the administrative division of Houet, 
Hauts-Bassins, Burkina Faso.

Collections were made in July 2012.



Samples and genomic data originate from the following studies

,,,Sample set
Contributor,Study,Data release,
Austin Burt,Ag1000G,Ag3.0,AG1000G-BF-A


## Sampling information
### Collection locations

In [ ]:
from ipyleaflet import Map, Marker, basemaps

center = cohort[['latitude', 'longitude']].to_list()
m = Map(center=center, zoom=9, basemap=basemaps.OpenTopoMap)

for coh_id, row in df_samples.iterrows():
    lat, long = row[['latitude', 'longitude']]
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'

    marker = Marker(location=(lat, long), draggable=True, opacity=0.7, color=color)
    m.add_layer(marker);

display(m)

### Collection dates

In [ ]:
if month_collections:
    fig = px.bar(df_collection_dates, x='month', y='count', title=f"Collection dates {quarter} {year}")
    display(fig)
else:
    display(Markdown("No per-month collection data exists for this cohort."))

## Selection scans

In [ ]:
# load window sizes 
calibration_dir = "build/h12-calibration"
with open(here() / calibration_dir / f"{cohort_id}.yaml") as calibration_file:
    calibration_params = yaml.safe_load(calibration_file)
window_size = calibration_params["h12_window_size"]

if cohort.taxon == 'arabiensis':
    phasing_analysis = 'arab'
else:
    phasing_analysis = 'gamb_colu'

ihs_window_size = 100

def plot_h12_g123_ihs_tracks(
        contig, 
        window_size, 
        ihs_window_size,
        phasing_analysis, 
        sample_sets, 
        sample_query, 
        min_cohort_size,
        max_cohort_size,
        sizing_mode='stretch_width', 
        show=False, 
        width=800, 
        genes_height=100
    ):

    fig1 = ag3.plot_h12_gwss_track(
        contig=contig, 
        window_size=window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        show=show,
        width=width,
    )
    fig1.xaxis.visible = False

    fig2 = ag3.plot_g123_gwss_track(
        contig=contig, 
        # TODO Calibrate G123 window size separately?
        window_size=window_size, 
        site_mask=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig2.xaxis.visible = False

    fig3 = ag3.plot_ihs_gwss_track(
        contig=contig, 
        window_size=ihs_window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig3.xaxis.visible = False


    fig4 = ag3.plot_genes(
        region=contig, 
        show=show,
        sizing_mode=sizing_mode,
        width=width,
        height=genes_height,
        x_range=fig1.x_range
        )
                        
    fig = bklay.gridplot(
        [fig1, fig2, fig3, fig4],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode=sizing_mode,
    )
    return fig 

In [ ]:
for contig in contigs:
    
    display(HTML(f"<h3>Chromosome {contig}</h3>"))
    
    fig = plot_h12_g123_ihs_tracks(
        contig=contig,
        window_size=window_size,
        ihs_window_size=ihs_window_size,
        phasing_analysis=phasing_analysis,
        sample_sets=sample_sets,
        sample_query=cohort['sample_query'],
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
    );

    bkplt.show(fig)


## Diagnostics
### H12 calibration

In [ ]:
window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)

ag3.plot_h12_calibration(
    contig=h12_calibration_contig,
    analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort['sample_query'],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

### G123 Calibration

In [ ]:
# ag3.plot_g123_calibration(
#     contig=h12_calibration_contig,
#     sites=phasing_analysis,
#     site_mask=phasing_analysis,
#     sample_sets=sample_sets,
#     sample_query=cohort['sample_query'],
#     min_cohort_size=min_cohort_size,
#     max_cohort_size=max_cohort_size,
#     window_sizes=window_sizes,
# );

In [ ]:
print("TODO")